In [1]:
import numpy as np
import cv2
from tensorflow.lite.python.interpreter import Interpreter

def rotated_Rectangle(img, rotatedRect, color, thickness=1, lineType=cv2.LINE_8, shift=0):
    (x, y), (width, height), angle = rotatedRect
    pt1_1 = (int(x + width / 2), int(y + height / 2))
    pt2_1 = (int(x + width / 2), int(y - height / 2))
    pt3_1 = (int(x - width / 2), int(y - height / 2))
    pt4_1 = (int(x - width / 2), int(y + height / 2))

    t = np.array([[np.cos(angle),   -np.sin(angle), x-x*np.cos(angle)+y*np.sin(angle)],
                  [np.sin(angle), np.cos(angle),  y-x*np.sin(angle)-y*np.cos(angle)],
                  [0,             0,              1]])

    tmp_pt1_1 = np.array([[pt1_1[0]], [pt1_1[1]], [1]])
    tmp_pt1_2 = np.dot(t, tmp_pt1_1)
    pt1_2 = (int(tmp_pt1_2[0][0]), int(tmp_pt1_2[1][0]))

    tmp_pt2_1 = np.array([[pt2_1[0]], [pt2_1[1]], [1]])
    tmp_pt2_2 = np.dot(t, tmp_pt2_1)
    pt2_2 = (int(tmp_pt2_2[0][0]), int(tmp_pt2_2[1][0]))

    tmp_pt3_1 = np.array([[pt3_1[0]], [pt3_1[1]], [1]])
    tmp_pt3_2 = np.dot(t, tmp_pt3_1)
    pt3_2 = (int(tmp_pt3_2[0][0]), int(tmp_pt3_2[1][0]))

    tmp_pt4_1 = np.array([[pt4_1[0]], [pt4_1[1]], [1]])
    tmp_pt4_2 = np.dot(t, tmp_pt4_1)
    pt4_2 = (int(tmp_pt4_2[0][0]), int(tmp_pt4_2[1][0]))

    points = np.array([pt1_2, pt2_2, pt3_2, pt4_2])
    return points

def non_max_suppression(boxes, probs=None, angles=None, overlapThresh=0.3):
    if len(boxes) == 0:
        return [], []

    if boxes.dtype.kind == "i":
        boxes = boxes.astype("float")

    pick = []
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]

    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = y2

    if probs is not None:
        idxs = probs

    idxs = np.argsort(idxs)

    while len(idxs) > 0:
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

        xx1 = np.maximum(x1[i], x1[idxs[:last]])
        yy1 = np.maximum(y1[i], y1[idxs[:last]])
        xx2 = np.minimum(x2[i], x2[idxs[:last]])
        yy2 = np.minimum(y2[i], y2[idxs[:last]])

        w = np.maximum(0, xx2 - xx1 + 1)
        h = np.maximum(0, yy2 - yy1 + 1)

        overlap = (w * h) / area[idxs[:last]]
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

    return boxes[pick].astype("int"), angles[pick]

def decode_predictions(scores, geometry):
    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []
    angles = []

    for y in range(0, numRows):
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        for x in range(0, numCols):
            if scoresData[x] < 0.5:
                continue

            (offsetX, offsetY) = (x * 4.0, y * 4.0)
            angle = anglesData[x]
            cos = np.cos(angle)
            sin = np.sin(angle)

            h = xData0[x] + xData2[x]
            w = xData1[x] + xData3[x]

            endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
            endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
            startX = int(endX - w)
            startY = int(endY - h)

            rects.append((startX, startY, endX, endY))
            confidences.append(scoresData[x])
            angles.append(angle)

    return (rects, confidences, angles)


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.lite.python.interpreter import Interpreter

# Функция для получения пар (изображение, аннотация)
def get_image_annotation_pairs(image_folder, annotation_folder):
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')])
    annotation_files = sorted([f for f in os.listdir(annotation_folder) if f.endswith('.txt')])

    assert len(image_files) == len(annotation_files), "Количество изображений и аннотаций не совпадает!"

    pairs = []
    for img, ann in zip(image_files, annotation_files):
        pairs.append((os.path.join(image_folder, img), os.path.join(annotation_folder, ann)))

    return pairs

# Функция IoU для сравнения прямоугольников
def compute_iou(box1, box2):
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    box1Area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2Area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    iou = interArea / float(box1Area + box2Area - interArea)
    
    return iou

# Функция для проверки попадания предсказанных боксов в аннотации
def compare_boxes(predicted_boxes, true_boxes, iou_threshold=0.5):
    TP, FP, FN = 0, 0, 0
    matched = set()

    for pred_box in predicted_boxes:
        found_match = False
        for i, true_box in enumerate(true_boxes):
            iou = compute_iou(pred_box, true_box)
            if iou >= iou_threshold:
                TP += 1
                matched.add(i)
                found_match = True
                break
        if not found_match:
            FP += 1

    FN = len(true_boxes) - len(matched)
    return TP, FP, FN

# Функция для загрузки аннотаций
def load_annotations(annotation_path):
    annotations = []
    with open(annotation_path, 'r', encoding='utf-8-sig') as file:
        for line in file:
            line = line.strip().replace('﻿', '')  
            parts = line.split(',')
            if len(parts) >= 9:
                try:
                    coords = list(map(int, parts[:8]))
                    text = parts[8]  
                    annotations.append({
                        "coords": np.array(coords).reshape(4, 2),
                        "text": text
                    })
                except ValueError:
                    print(f"Ошибка в аннотации: {line}")
    return annotations

# Инициализация модели EAST
model_path = 'east_text_detection_320x320_integer_quant.tflite'
interpreter = Interpreter(model_path=model_path, num_threads=4)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Создаем папку для сохранения результатов
output_folder = "output_images"
os.makedirs(output_folder, exist_ok=True)

# Получаем пары (изображение, аннотация)
test_pairs = get_image_annotation_pairs(
    'data/ch4_test_images',
    'ch4_test_localization_transcription_gt'
)

# Переменные для подсчета метрик
total_TP, total_FP, total_FN = 0, 0, 0

# Обработка изображений
for img_path, ann_path in test_pairs:
    img = cv2.imread(img_path)
    if img is None:
        print(f"Ошибка загрузки изображения: {img_path}")
        continue

    true_annotations = load_annotations(ann_path)
    original_height, original_width, _ = img.shape

    # Подготовка изображения
    image_resized = cv2.resize(img, (320, 320))
    image_resized = image_resized.astype(np.float32)
    mean = np.mean(image_resized, axis=(0, 1), keepdims=True)
    image_resized -= mean
    image_resized = cv2.cvtColor(image_resized, cv2.COLOR_BGR2RGB)
    image_resized = np.expand_dims(image_resized, axis=0)

    interpreter.set_tensor(input_details[0]['index'], image_resized)
    interpreter.invoke()

    scores = interpreter.get_tensor(output_details[0]['index'])
    geometry = interpreter.get_tensor(output_details[1]['index'])

    # Декодируем предсказания
    scores = np.transpose(scores, [0, 3, 1, 2])
    geometry = np.transpose(geometry, [0, 3, 1, 2])
    (rects, confidences, angles) = decode_predictions(scores, geometry)
    boxes, angles = non_max_suppression(np.array(rects), probs=confidences, angles=np.array(angles))

    # Приводим боксы к исходному размеру изображения
    height_scale = original_height / 320
    width_scale = original_width / 320
    predicted_boxes = []
    for ((startX, startY, endX, endY), angle) in zip(boxes, angles):
        startX, startY = int(startX * width_scale), int(startY * height_scale)
        endX, endY = int(endX * width_scale), int(endY * height_scale)
        width, height = abs(endX - startX), abs(endY - startY)
        centerX, centerY = int(startX + width / 2), int(startY + height / 2)
        rotatedRect = ((centerX, centerY), (width, height), -angle)
        box = cv2.boxPoints(rotatedRect)
        box = np.int32(box)
        predicted_boxes.append([startX, startY, endX, endY])
        cv2.polylines(img, [box], isClosed=True, color=(255, 0, 0), thickness=2)

    true_boxes = []
    for ann in true_annotations:
        coords = ann["coords"].flatten()
        x_min, x_max = min(coords[0::2]), max(coords[0::2])
        y_min, y_max = min(coords[1::2]), max(coords[1::2])
        true_boxes.append([x_min, y_min, x_max, y_max])
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)


    # Сравниваем боксы
    TP, FP, FN = compare_boxes(predicted_boxes, true_boxes)
    total_TP += TP
    total_FP += FP
    total_FN += FN

    # Рисуем боксы
    for (x_min, y_min, x_max, y_max) in true_boxes:
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)  # Истинные боксы (зеленые)

    for (x_min, y_min, x_max, y_max) in predicted_boxes:
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)  # Предсказанные боксы (синие)

    # Сохраняем изображение
    output_path = os.path.join(output_folder, os.path.basename(img_path))
    cv2.imwrite(output_path, img)


# Выводим итоговые метрики
print(f"Total TP: {total_TP}")
print(f"Total FP: {total_FP}")
print(f"Total FN: {total_FN}")

# Выводим итоговые метрики
precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) > 0 else 0
recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) > 0 else 0
f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1_score:.4f}")


Total TP: 408
Total FP: 741
Total FN: 4822
Precision: 0.3551
Recall: 0.0780
F1-Score: 0.1279
